In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

In [2]:
#Loading Fashion-MNIST Dataset
fashion_mnist = keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
#Normalizing Dataset Images

x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
train_images_norm = x_train / 255.0
test_images_norm = x_test / 255.0


In [4]:
#Defining the Model


model = Sequential()
model.add(Conv2D(filters=48, kernel_size=4,  activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=42, kernel_size=2, activation='relu')) 
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(10))
# Print the model summary
model.summary()

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 48)        816       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 42)        8106      
_________________________________________________________________
flatten (Flatten)            (None, 5082)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                162656    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 171,908
Trainable params: 171,908
Non-trainable params: 0
__________________________________________________

In [5]:
#Training of the Model
model.fit(train_images_norm, y_train, epochs=1, validation_split=0.1)

1688/1688 [==============================] - 16s 6ms/step - loss: 0.6417 - accuracy: 0.7742 - val_loss: 0.3322 - val_accuracy: 0.8772


In [6]:
#Save the Model to apply Quantization

MODELS_DIR = '/home/wigand/projects/ml/models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [7]:
model.save(MODEL_TF)

INFO:tensorflow:Assets written to: /home/wigand/projects/ml/models/model/assets


In [8]:
# Method to get representative Dataset for Integer Quantization
def representative_dataset():
  for i in range(500):
    yield([train_images_norm.astype(np.float32)[i].reshape(1, 28, 28, 1)])

In [9]:
# Convert the model to the TensorFlow Lite format with Quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
# Set the optimization flag
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_QUANT_TFLITE, "wb").write(model_quant_tflite)

177864

In [10]:
def predict_tflite(tflite_model, x_test):
    # Prepare the test data
    x_test_ = x_test.copy()
    x_test_ = x_test_.reshape((10000, 28, 28, 1))
    x_test_ = x_test_.astype(np.float32)

    # Initialize the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    input_scale, input_zero_point = input_details["quantization"]
    if (input_scale, input_zero_point) != (0.0, 0):
        x_test_ = x_test_ / input_scale + input_zero_point
        x_test_ = x_test_.astype(input_details["dtype"])

    # Invoke the interpreter
    y_pred = np.empty(shape= (10000, 10), dtype=output_details["dtype"])
    print(input_details['dtype'])
    for i in range(len(x_test_)):
        interpreter.set_tensor(input_details["index"], [x_test_[i]])
        interpreter.invoke()
        inter = interpreter.get_tensor(output_details["index"])
        y_pred[i] = np.array(inter[0], dtype="float32")

    # If required, dequantized the output layer (from integer to float)
    output_scale, output_zero_point = output_details["quantization"]
    if (output_scale, output_zero_point) != (0.0, 0):
        y_pred = y_pred.astype(np.float32)
        y_pred = (y_pred - output_zero_point) * output_scale

    return y_pred

def evaluate_tflite(tflite_model, x_test, y_true):
    global model
    y_pred = predict_tflite(tflite_model, x_test)
    loss_function = tf.keras.losses.get(model.loss)
    loss = loss_function(y_true, y_pred).numpy()
    return loss

In [11]:
# Make predictions for the Test Set
y_test_pred_tf = model.predict(test_images_norm)
y_test_pred_quant_tflite = predict_tflite(model_quant_tflite, test_images_norm)

<class 'numpy.float32'>


In [12]:
# Take the category with the highest Score
y_last = np.empty(10000, dtype="int8")
for g in range(y_test_pred_quant_tflite.shape[0]):
    y_last[g] = np.argmax(y_test_pred_quant_tflite[g])

In [13]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_last, y_test))

0.8737


In [14]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_QUANT_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_QUANT_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

!sed -i 's/unsigned char/#include "model.h"\nalignas(8) const unsigned char/' {MODEL_TFLITE_MICRO}
!sed -i 's/unsigned int g_model_len/const int g_model_len/' {MODEL_TFLITE_MICRO}

Paketlisten werden gelesen... Fertig
E: Sperrdatei /var/lib/apt/lists/lock konnte nicht geöffnet werden. - open (13: Keine Berechtigung)
E: Das Verzeichnis /var/lib/apt/lists/ kann nicht gesperrt werden.
W: Problem beim Entfernen (unlink) der Datei /var/cache/apt/pkgcache.bin - RemoveCaches (13: Keine Berechtigung)
W: Problem beim Entfernen (unlink) der Datei /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Keine Berechtigung)


In [15]:
# Uncomment to print the C source file 

#!cat {MODEL_TFLITE_MICRO}


In [16]:
# Move the created C-Array of the Model to the Arduino Sketch Folder

import os
import shutil
os.rename(MODEL_TFLITE_MICRO, 'model.cpp')
shutil.move("/home/wigand/projects/ml/model.cpp", "/home/wigand/Arduino/Modell_V1/model.cpp")

'/home/wigand/Arduino/Modell_V1/model.cpp'

In [17]:
#Print Prediction of Desktop Inference

y_test_pred_quant_tflite[1878]

array([ 2.1396892, -2.6431458,  7.5518446, -1.7620971,  4.0276504,
       -2.3914173,  5.9156113, -9.565669 , -0.5034563, -6.544932 ],
      dtype=float32)